In [1]:
pip install pyspark==3.2.4

Defaulting to user installation because normal site-packages is not writeable
  Using cached pyspark-3.2.4.tar.gz (281.5 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
  Created wheel for pyspark: filename=pyspark-3.2.4-py2.py3-none-any.whl size=282040957 sha256=1ab626d1eafd90ae38117370ae3b60f5e95ce0d15c97a9aac04df000930cf693
  Stored in directory: c:\users\saksh\appdata\local\pip\cache\wheels\bf\f6\18\acaa11d057c23e749eea1773c9516d58fbc8ddcdd1492ad3fd
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Can't uninstall 'py4j'. No files were found to uninstall.
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.4.0
    Can't uninstall 'pyspark'. No files were found to uninstall.
Note: you may need to restart the kernel to use updated packages.


In [23]:
from pyspark.sql import SparkSession

In [24]:
if __name__ =="__main__":

    spark = SparkSession \
    .builder \
    .appName("maarketdata") \
    .master("local[1]") \
    .getOrCreate()
    
    schema = "age INT, job STRING , marital STRING , education STRING, default STRING, balance INT,\
     housing STRING, loan STRING, contact STRING, day INT, month STRING, \
    duration INT, campaign INT, pdays INT, previous INT, poutcome STRING, y STRING "

    data = spark.read.csv("D:/Projectssssss/Capstone Project/marketanalysisdata.csv",
                          schema=schema,sep=",",header=True)
#     selected_data = data.select("age","job","marital","education")

    data.printSchema()
    data.show()
    

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+---

In [13]:
dataList = [("Java", 20000), ("Python", 100000), ("Scala", 3000)]
rdd=spark.sparkContext.parallelize(dataList)

In [14]:
type(rdd)

pyspark.rdd.RDD

In [19]:
data.createOrReplaceTempView("sample")
df2=spark.sql("select * from sample where age > 40 order by age ")
df2.printSchema()
df2.show()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+---

In [50]:
# giving marketing success rate

df3 = spark.sql("select y,count(*) as cnt from sample group by y")
df4 = spark.sql("select count(*) as total_cnt from sample")
df3.show()
df4.show()

df4.createOrReplaceTempView("total")
df3.createOrReplaceTempView("market")

+---+-----+
|  y|  cnt|
+---+-----+
| no|39922|
|yes| 5289|
+---+-----+

+---------+
|total_cnt|
+---------+
|    45211|
+---------+



In [55]:
# market success rate and failure 
df5=spark.sql("select m.y,m.cnt , t.total_cnt-m.cnt, m.cnt/t.total_cnt * 100 as percent from market m left join total t ")
df5.show()

+---+-----+-----------------+------------------+
|  y|  cnt|(total_cnt - cnt)|           percent|
+---+-----+-----------------+------------------+
| no|39922|             5289| 88.30151954170445|
|yes| 5289|            39922|11.698480458295547|
+---+-----+-----------------+------------------+



In [56]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [58]:
df6 = spark.sql("select max(age) as Max_age, min(age) as Min_age, Avg(age) as mean_age from sample where y='yes'")
df6.show()

+-------+-------+------------------+
|Max_age|Min_age|          mean_age|
+-------+-------+------------------+
|     95|     18|41.670069956513515|
+-------+-------+------------------+



In [59]:
df7 = spark.sql("select avg(balance) as mean_balance,percentile_approx(balance,0.5) as median_balance from sample ").show()

+------------------+--------------+
|      mean_balance|median_balance|
+------------------+--------------+
|1362.2720576850766|           448|
+------------------+--------------+



In [62]:
df8 = spark.sql("select age,count(*) as cnt from sample where y ='yes' group by age order by cnt").show()

+---+---+
|age|cnt|
+---+---+
| 95|  1|
| 93|  2|
| 92|  2|
| 90|  2|
| 87|  3|
| 85|  4|
| 86|  4|
| 84|  5|
| 81|  6|
| 83|  6|
| 18|  7|
| 82|  8|
| 79| 10|
| 19| 11|
| 80| 12|
| 74| 13|
| 78| 14|
| 20| 15|
| 75| 15|
| 76| 16|
+---+---+
only showing top 20 rows



In [63]:
df9 = spark.sql("select marital,count(*)as cnt from sample where y = 'yes' group by marital order by cnt").show()

+--------+----+
| marital| cnt|
+--------+----+
|divorced| 622|
|  single|1912|
| married|2755|
+--------+----+



In [64]:
df10 = spark.sql("select age, marital,count(*) as cnt,sum(case when y = 'yes' then 1 else 0 end) as deposit from sample group by age, marital order by age, marital").show()

+---+--------+---+-------+
|age| marital|cnt|deposit|
+---+--------+---+-------+
| 18|  single| 12|      7|
| 19|  single| 35|     11|
| 20| married|  3|      1|
| 20|  single| 47|     14|
| 21| married|  5|      1|
| 21|  single| 74|     21|
| 22| married|  9|      0|
| 22|  single|120|     40|
| 23| married| 27|      2|
| 23|  single|175|     42|
| 24|divorced|  1|      0|
| 24| married| 53|     10|
| 24|  single|248|     58|
| 25|divorced|  6|      0|
| 25| married| 98|     14|
| 25|  single|423|     99|
| 26|divorced| 20|      0|
| 26| married|170|     13|
| 26|  single|615|    121|
| 27|divorced| 18|      2|
+---+--------+---+-------+
only showing top 20 rows



In [73]:
df11 = spark.sql("select * ,case when age<=20 then '20-' \
                  when age >20 and age<=30 then '20-30' \
                  when age >30 and age<=40 then '30-40' \
                  when age >40 and age<=50 then '40-50' \
                  when age >50 and age<=60 then '50-60' \
                  when age >60 and age<=70 then '60-70' \
                     else '70+' end as age_featuring from sample").show()



+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+-------------+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|age_featuring|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+-------------+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|        50-60|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|        40-50|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|        30-40|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|

In [72]:
df11.createOrReplaceTempView("aged")
df12 = spark.sql("select age_featuring from aged group by age_featuring").show()

AttributeError: 'NoneType' object has no attribute 'createOrReplaceTempView'

In [1]:
print(5)

5


In [11]:
schema2  = "id INT, gender STRING, marks INT"
dat = spark.read.csv("D:/Projectssssss/Capstone Project/ct.csv",
                          schema=schema2,sep=",",header=True)

In [12]:
dat.createOrReplaceTempView("window")
dat.show()

+---+------+-----+
| id|gender|marks|
+---+------+-----+
|  2|     m|   50|
|  3|     m|   70|
|  4|     m|   80|
|  5|     f|   20|
|  6|     f|   30|
|  7|     f|   30|
|  8|     f|   40|
+---+------+-----+



In [14]:
dats = spark.sql("select *, row_number() over (partition by gender order by marks)as row_num from window ")
dats.show()

+---+------+-----+-------+
| id|gender|marks|row_num|
+---+------+-----+-------+
|  5|     f|   20|      1|
|  6|     f|   30|      2|
|  7|     f|   30|      3|
|  8|     f|   40|      4|
|  2|     m|   50|      1|
|  3|     m|   70|      2|
|  4|     m|   80|      3|
+---+------+-----+-------+



In [15]:
dats2 = spark.sql("select *, rank() over (partition by gender order by marks)as rank from window ")
dats2.show()

+---+------+-----+----+
| id|gender|marks|rank|
+---+------+-----+----+
|  5|     f|   20|   1|
|  6|     f|   30|   2|
|  7|     f|   30|   2|
|  8|     f|   40|   4|
|  2|     m|   50|   1|
|  3|     m|   70|   2|
|  4|     m|   80|   3|
+---+------+-----+----+



In [16]:
dats3 = spark.sql("select *, dense_rank() over (partition by gender order by marks)as dense_rank from window ")
dats3.show()

+---+------+-----+----------+
| id|gender|marks|dense_rank|
+---+------+-----+----------+
|  5|     f|   20|         1|
|  6|     f|   30|         2|
|  7|     f|   30|         2|
|  8|     f|   40|         3|
|  2|     m|   50|         1|
|  3|     m|   70|         2|
|  4|     m|   80|         3|
+---+------+-----+----------+



In [20]:
# dats3.createOrReplaceTempView("sub")
# dats4 = spark.sql("select gender,max(dense_rank) as rank from sub group by gender")
# dats4.show()



AnalysisException: expression 'sub.id' is neither present in the group by, nor is it an aggregate function. Add to group by or wrap in first() (or first_value) if you don't care which value you get.;
Aggregate [gender#361], [id#360, gender#361, max(dense_rank#443) AS rank#501]
+- SubqueryAlias sub
   +- View (`sub`, [id#360,gender#361,marks#362,dense_rank#443])
      +- Project [id#360, gender#361, marks#362, dense_rank#443]
         +- Project [id#360, gender#361, marks#362, dense_rank#443, dense_rank#443]
            +- Window [dense_rank(marks#362) windowspecdefinition(gender#361, marks#362 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS dense_rank#443], [gender#361], [marks#362 ASC NULLS FIRST]
               +- Project [id#360, gender#361, marks#362]
                  +- SubqueryAlias window
                     +- View (`window`, [id#360,gender#361,marks#362])
                        +- Relation [id#360,gender#361,marks#362] csv


In [25]:
data.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [26]:
newd = spark.read.csv("D:/Projectssssss/Capstone Project/marketanalysisdata.csv",header=True)
newd.printSchema()

root
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [45]:
newd2 =newd.withColumnRenamed("age","Age")
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

In [52]:
newd2.printSchema()
newd3 = newd2.withColumn("Age",col("Age").cast(IntegerType()))

root
 |-- Age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [50]:
newd2.filter((newd2.marital=='married') &(newd2.job=='management')).count()

5400